In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from datetime import date
import time

import pandas as pd

service = Service(ChromeDriverManager().install())
options = Options()

In [2]:
rota_site = 'https://integrasus.saude.ce.gov.br/#/indicadores/indicadores-vigilancia-saude/cenario-arbovirose'

rota_btn_2024 = '/html/body/div[3]/div[2]/div/div/div/mat-option[1]/span'
rota_btn_2023 = '/html/body/div[3]/div[2]/div/div/div/mat-option[2]/span'

today = date.today()

In [3]:
# Opcao para executar comandos sem abrir o navegador
options.add_argument("--headless=new")
driver = webdriver.Chrome(service=service, options=options)

driver.get(rota_site)

time.sleep(5)

print('Obtendo cabeçalho...')
#Extrai cabeçalho
header = []

for c in range(1, 8):
    xpath_item = f'/html/body/int-root/div/mat-sidenav-container/mat-sidenav-content/int-cenario-arboviroses/div/div[7]/int-cenario-arbovirose-mapa-tabela/div/div[2]/div[2]/table/thead/tr/th[{c}]'
    item = driver.find_element(By.XPATH, xpath_item)
    item_text = item.text.strip()
    header.append(item_text)

header_2023 = header
header_2023.pop(6)

print('Cabeçalho obtido.')

# Selecionar dados 2023
menu_btn = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.ID, "mat-select-0"))
)

menu_btn.click()
time.sleep(1)

btn_2023 = driver.find_element(By.XPATH, rota_btn_2023)
btn_2023.click()

print('Selecionado o ano: 2023')

time.sleep(5)

linha_tabela = driver.find_elements(By.CSS_SELECTOR, 'tbody tr')

# Tabela de dados
tabela_info_2023 = []

for linha in linha_tabela:
    itens_linha = []

    item_linha = linha.find_elements(By.TAG_NAME ,'td')
    
    for item in item_linha:
        itens_linha.append(item.text.strip())
    
    tabela_info_2023.append(itens_linha)

# Selecionar dados 2024
menu_btn = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.ID, "mat-select-0"))
)

menu_btn.click()
time.sleep(1)

btn_2023 = driver.find_element(By.XPATH, rota_btn_2024)
btn_2023.click()

print('Selecionado o ano: 2024')

time.sleep(5)

linha_tabela = driver.find_elements(By.CSS_SELECTOR, 'tbody tr')

# Tabela de dados
tabela_info_2024 = []

for linha in linha_tabela:
    itens_linha = []

    item_linha = linha.find_elements(By.TAG_NAME ,'td')
    
    for item in item_linha:
        itens_linha.append(item.text.strip())
    
    tabela_info_2024.append(itens_linha)

print('Extração de dados finalizada.')
time.sleep(1)

print('Extração de cabeçalho finalizada. Fechando navegador.')

driver.quit()

Obtendo cabeçalho...
Cabeçalho obtido.
Selecionado o ano: 2023
Selecionado o ano: 2024
Extração de dados finalizada.
Extração de cabeçalho finalizada. Fechando navegador.


In [4]:
# Ajuste para atindir mesmo leiaute
for i in tabela_info_2024:
    i.pop(6)

In [11]:
# Dicionario com tipagem ajustada e adicao de informação de mesorregiao - ano 2023
dict_info_2023 = []

for linha in tabela_info_2023:
    dict_info_2023.append({
        header[0]: linha[0],
        header[1]: int(linha[1].replace('.','')),
        header[2]: int(linha[2]),
        header[3]: int(linha[3]),
        header[4]: int(linha[4]),
        header[5]: float(linha[5])
    })

In [6]:
# Dicionario com tipagem ajustada e adicao de informação de mesorregiao - ano 2024
dict_info_2024 = []

for linha in tabela_info_2024:
    dict_info_2024.append({
        header[0]: linha[0],
        header[1]: int(linha[1].replace('.','')),
        header[2]: int(linha[2]),
        header[3]: int(linha[3]),
        header[4]: int(linha[4]),
        header[5]: float(linha[5])
    })

In [23]:
# Geração de dataframes para exportação
df_info_2023 = pd.DataFrame(dict_info_2023)
df_info_2024 = pd.DataFrame(dict_info_2024)

# Extração de dados para arquivos csv
df_info_2023.to_csv(f'Dados_IntegraSUS_2023_{today}.csv', index=False, sep='|')
df_info_2024.to_csv(f'Dados_IntegraSUS_2024_{today}.csv', index=False, sep='|')

In [24]:
# calcular os totais por municipio
def calcular_totais(infos, info_por_municipio):
    for dado in infos:
        municipio = dado['Município']
        if municipio not in info_por_municipio:
            info_por_municipio[municipio] = {'Notificados': 0, 'Casos prováveis': 0, 'Confirmados': 0}
        info_por_municipio[municipio]['Notificados'] += dado['Notificados']
        info_por_municipio[municipio]['Casos prováveis'] += dado['Casos prováveis']
        info_por_municipio[municipio]['Confirmados'] += dado['Confirmados']

In [25]:
# armazenar os dados totais
info_por_municipio_2023 = {}
info_por_municipio_2024 = {}

calcular_totais(dict_info_2023, info_por_municipio_2023)
calcular_totais(dict_info_2024, info_por_municipio_2024)

In [26]:
# Calculo da variação percentual
resultado_percent = {}

for municipio in info_por_municipio_2023:
    dados_resultado = {}
    for caso in ['Notificados', 'Casos prováveis', 'Confirmados']:    
        total_2023 = info_por_municipio_2023[municipio][caso]
        total_2024 = info_por_municipio_2024[municipio][caso]
        variacao_percentual = ((total_2024 - total_2023) / total_2023) * 100 if total_2023 != 0 else 0
        dados_resultado[caso] = f'{variacao_percentual:.2f}'
    resultado_percent[municipio] = dados_resultado

In [27]:
# Ajuste para criação de dataframe
info_percent = []

for municipio, variacoes_municipio in resultado_percent.items():
    linha = []
    
    linha.append(municipio)

    for caso, variacao_percentual in variacoes_municipio.items():
        linha.append(variacao_percentual)

    info_percent.append({
        'Município %': linha[0],
        'Notificados %': linha[1], 
        'Casos prováveis %': linha[2], 
        'Confirmados %': linha[3]
    })

In [28]:
# Geração de dataframes para exportação
df_resultado_percent = pd.DataFrame(info_percent)

# Extração de dados para arquivos csv
df_resultado_percent.to_csv(f'Resultado_Percent_IntegraSUS_{today}.csv', index=False, sep='|')

In [29]:
# ajuste de nomes de colunas dos dataframes
df_info_2023 = df_info_2023.rename(columns={'Notificados': 'Notificados 2023', 'Casos prováveis': 'Casos prováveis 2023', 'Confirmados': 'Confirmados 2023', 'Incid. acumulada dos prováveis (ano)': 'Incid. acumulada dos prováveis (ano) 2023'})
df_info_2024 = df_info_2024.rename(columns={'Município': 'Município 2024', 'Habitantes': 'Habitantes 2024', 'Notificados': 'Notificados 2024', 'Casos prováveis': 'Casos prováveis 2024', 'Confirmados': 'Confirmados 2024', 'Incid. acumulada dos prováveis (ano)': 'Incid. acumulada dos prováveis (ano) 2024'})

In [33]:
df_concat = pd.concat([df_info_2023, df_info_2024, df_resultado_percent], axis=1)

In [35]:
df_concat.drop(columns=['Município 2024','Habitantes 2024','Município %'])

,Município,Habitantes,Notificados 2023,Casos prováveis 2023,Confirmados 2023,Incid. acumulada dos prováveis (ano) 2023,Notificados 2024,Casos prováveis 2024,Confirmados 2024,Incid. acumulada dos prováveis (ano) 2024,Notificados %,Casos prováveis %,Confirmados %
0,ABAIARA,10038,18,7,7,69.7,41,32,20,318.8,127.78,357.14,185.71
1,ACARAPE,14027,13,1,0,7.1,11,6,0,42.8,-15.38,500.00,0.00
2,ACARAU,64806,186,16,14,24.7,84,7,2,10.8,-54.84,-56.25,-85.71
3,ACOPIARA,44962,45,6,6,13.3,35,10,2,22.2,-22.22,66.67,-66.67
4,AIUABA,14076,32,17,13,120.8,21,6,0,42.6,-34.38,-64.71,-100.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,URUBURETAMA,20189,29,1,1,5.0,22,10,0,49.5,-24.14,900.00,-100.00
180,URUOCA,13746,5,1,1,7.3,35,2,1,14.6,600.00,100.00,0.00
181,VARJOTA,18105,26,4,2,22.1,95,71,0,392.2,265.38,1675.00,-100.00
182,VARZEA ALEGRE,38984,65,5,5,12.8,91,30,6,77.0,40.00,500.00,20.00


In [36]:
# Extração de dados para arquivos csv
df_concat.to_csv(f'Dados_Concat_IntegraSUS_{today}.csv', index=False, sep='|')